to know how many cases are in this file

In [66]:
import fitz
import re

def extract_target_number(pdf_path):
    doc = fitz.open(pdf_path)

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if (
                            'number' in b and b['number'] == 5
                            and 'size' in span and span['size'] == 12.0
                            and 'font' in span and span['font'] == 'Helvetica-Bold'
                            and page_num == 0  # Only on the first page
                        ):
                            match = re.search(r'\((\d+)\)', span["text"])
                            if match:
                                return int(match.group(1))

    doc.close()


get the title list

In [68]:
import fitz
import re

def extract_title(pdf_path):
    doc = fitz.open(pdf_path)
    extracted_text_list = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]

        for b in blocks:
            if "lines" in b:
                for line in b["lines"]:
                    for span in line["spans"]:
                        if 'font' in span and span['font'] == 'Helvetica-Oblique':
                            result_string = re.sub(r',[^,]*$', '', span["text"].strip())
                            extracted_text_list.append(result_string)

        if len(extracted_text_list) >= extract_target_number(pdf_path):
            break

    doc.close()
    return extracted_text_list


In [69]:

pdf_path = "D:/antitrust/verdict/Federal Circuit (30).PDF"
# pdf_path = "D:/antitrust/verdict/Files (100).PDF"
result_list = extract_title(pdf_path)

In [70]:
len(result_list)

31

get other information and write them into csv

In [71]:
import fitz
import csv

def getAndWrite(pdf_path, case_list, output_csv):
    doc = fitz.open(pdf_path)
    matching_texts = []
    titles_list = set()

    title = None  # Variable to store 'number': 1 text
    court = None  # Variable to store 'number': 2 text
    date = None  # Variable to store 'number': 3 text
    no = None  # Variable to store 'number': 4 text
    consider_number_2 = False  # Flag to indicate whether to consider 'number': 2 text
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        blocks = page.get_text("dict")["blocks"]
        for b in blocks:
            if 'number' in b and b['number'] == 1:
                if "lines" in b:
                    # Reset text variables for each new 'number': 1
                    title = ""
                    court = None
                    date = None
                    no = None

                    for line in b["lines"]:
                        for span in line["spans"]:
                            if 'font' in span and span['font'] == 'Helvetica-BoldOblique':
                                title = span["text"].strip()
                                titles_list.add(title)
                                for case_name in case_list:
                                    if title.strip() == case_name.strip():
                                        consider_number_2 = True
                                        break  # Stop looking for 'number': 1 text once found
            if consider_number_2 and 'number' in b and b['number'] == 2:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                court = span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 3:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                date = date.strip() + span["text"].strip() if date else span["text"].strip()
            if consider_number_2 and 'number' in b and b['number'] == 4:
                if "lines" in b:
                    for line in b["lines"]:
                        for span in line["spans"]:
                            if ('font' in span and span['font'] == 'Helvetica') and ('size' in span and span['size'] == 10):
                                no = span["text"].strip()
                                consider_number_2 = False
                                break  # Stop looking for 'number': 4 text once found
        # Check if 'number': 1, 'number': 2, 'number': 3, and 'number': 4 text are present, then append to the result
        if no:
            matching_texts.append({'Title': title, 'Court': court, 'Date': date, 'No': no, 'page': page_num + 1 })

    doc.close()

    # Get the difference set
    difference_set = set(case_list).symmetric_difference(titles_list)
    print(difference_set)

    # Save matching_texts to a CSV file
    with open(output_csv, 'w', newline='', encoding='utf-8') as csv_file:
        fieldnames = ['Title', 'Court', 'Date', 'No', 'page']
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        
        # Write the header
        writer.writeheader()
        
        # Write the data
        for result in matching_texts:
            writer.writerow(result)


In [72]:
# Example usage:
pdf_path = "D:/antitrust/verdict/Federal Circuit (30).PDF"
# pdf_path = "D:/antitrust/verdict/Files (100).PDF"
case_list = result_list
output_csv = "output.csv"
getAndWrite(pdf_path, case_list, output_csv)

{'1', '9 to 5 Organization for Women Office Workers v. Board of Governors of', '9 to 5 Organization for Women Office Workers v. Board of Governors of Federal Reserve System'}
